Сделать

0) разделение на рус и каз

1) очищение стоп слов (предлоги, союзы и так далее)

2) лемматизация/стемминг?

3) применение fasttext full/compressed model и сравнение результатов с базовыми моделями

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import pandas as pd
from sklearn import linear_model
%matplotlib inline
from sklearn import svm
import matplotlib.pyplot as plt
import scipy.io as s
import sklearn.datasets as sd
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os


Подгружаем сет

In [2]:
df2 = pd.read_csv('only_you.csv',delimiter = ',')
df2.head()

,C1,C2,C3,C4,C5,C6,C7
0,0,22.01.15,150000011000008,1971,Ищанов А. и Угай А. 12 декабря 2011 года обрат...,1900003,Мошенничество
1,0,05.02.15,150000011000013,1971,"""УТВЕРЖДАЮ""\r\nНачальник 2-го Департамента\r\n...",1900003,Мошенничество
2,0,09.02.15,150000011000014,1971,08.06.2013г. в Талгарское РУВД ДВД Алматинской...,1900001,Мошенничество
3,0,26.02.15,150000011000019,1971,22.04.2011г. в ДБЭКП по Восточно-Казахстанской...,1900002,Мошенничество
4,0,19.05.15,150000011000027,1971,Изучением материалов по фактам якобы незаконно...,1900001,Мошенничество


In [3]:
kazakh = ['Қ','қ','Ұ','ұ','Ү','ү','Ө','ө','Ғ','ғ','Ң','ң','І','і','Ә','ә','$']
kazakh_zhyl=['жылы','жылдын','жылдың','ж.']
corr_bad = ['Ќ','ќ','Ў','ў','Ї','ї','Ґ','ґ','Є','є','Ѕ','ѕ','І','і','Ј','ј','?']
#naive kazakh language detector
converter = {}
for i in range(len(kazakh)):
    converter[corr_bad[i]] = kazakh[i]

def get_num(text):
    count=0
    for i in range(len(text)):
        if text[i] in kazakh:
            count=count+1
    return count/len(text)

def get_lang(text):
    for i in range(len(text)):
        if text[i] in kazakh:
            return 'kaz'
    return 'rus'
def get_lang_zhyl(text):
    for i in range(len(text)):
        if text[i] in kazakh_zhyl:
            return 'kaz'
    return 'rus'
def convert(text):
    res = ""
    for i in range(len(text)):
        if text[i] in converter:
            res +=  converter[text[i]]
        else:
            res += text[i]
    return res

In [4]:
#df2['language'] = df2['C5'].apply(lambda x: get_lang(x))

df2['text'] = df2['C5'].apply(lambda text: convert(text))

In [115]:
counter=0
df_1000=df2[counter:counter+1000]

In [116]:
df_1000['num']=df_1000['text'].apply(lambda text: get_num(text))

/home/alibi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [117]:
#evidence that messages are lost with only rus letters approach
df_zhyl_and_num=df_1000[(df_1000['num']==0) & (df_1000['text'].str.contains('жыл'))]
print(len(df_zhyl_and_num))
print(df_zhyl_and_num['text'].head())
print(df_zhyl_and_num['text'].values[6])

0
Series([], Name: text, dtype: object)


IndexError: index 6 is out of bounds for axis 0 with size 0

In [ ]:
#previously used and too broad both in kaz and rus
df_num=df_1000[df_1000['num']>0]
#too noisy because of rus abbreviations
df_z=df_1000[df_1000['text'].str.contains('ж.')]
#only kazakh messages for keyword search
df_zhyl_only=df_1000[df_1000['text'].str.contains('жыл')]
#possibly kazakh messages for keyword translation with (only russian letters with 'жыл') and ('ж.') capture
df_zhyl_or_num=df_1000[(df_1000['num']>0) | (df_1000['text'].str.contains('жыл'))]

In [118]:
print(len(df_num))
print(len(df_z))
#print(df_z['text'])
print(len(df_zhyl_or_num))
print(len(df_zhyl_only))
#print(df_zhyl_and_num['text'])
#print(df_zhyl_only['text'])

16314
45840
16848
7464


In [119]:


def normalize_text(row):
    """make text exmaple """
    txt = str(row['text'])
    print(txt)
    #return ' '.join(( txt ).split())



In [120]:
print(df_1000['text'].values[0])

Ищанов А. и Угай А. 12 декабря 2011 года обратились в ДВД Южно-Казахстанской области о принятии мер в отношении Заурбекова А. и других лиц, которые путем мошенничества завладели автомашиной марки "Тойота Прадо" 2009 года выпуска и денежными средствами в размере 6 800 000 тенге.
По данным доводам заявителей Аль-Фарабийским отделом полиции УВД г.Шымкент проведена доследственная проверка, по результатам которой 28 декабря 2011 года отказано в возбуждении уголовного дела за отсутствием состава преступления.
Прокуратурой Аль-Фарабийского района 13 января 2012 года указанное решение отменено.
	По результатам дополнительной проверки 23 января 2012 года УВД г.Шымкент возбуждено уголовное дело по п."б" ч.3 ст.177 УК.
	В последующем уголовное дело неоднократно приостанавливалось ввиду неустановления лица, подлежащего привлечению в качестве обвиняемого, а также прекращалось за отсутствием состава преступления.
	Последний раз постановлением старшего следователя УВД г.Шымкент Байбулекова С. от 23 и

In [121]:
#for i in range(len)
#df_1000['normalized'] = df_1000.apply(lambda row: normalize_text(row))

In [122]:
for i in range(len(df_1000)):
    df_1000['text'].values[i] = df_1000['text'].values[i].replace('\n', ' ').replace('\r', '')

In [123]:
filename='wrttmp.txt'


In [124]:
#clear previous contents
open(filename, 'w').close()
#write text here
for i in range(len(df_1000)):
    with open(filename, 'a') as out:
        out.write(df_1000['text'].values[i] + '\n')


In [145]:
#predict here
os.system('./fasttext predict langdetect.bin '+filename+' > tmp')
#open('tmp', 'r').read()

0

In [146]:
labels = pd.read_csv('tmp', header=None)
labels.head()

,0
0,__label__rus
1,__label__rus
2,__label__rus
3,__label__rus
4,__label__rus


In [147]:
ext = ['rus','kaz']

In [148]:
#print(labels[0][15])
#print(labels)
# classify tat as kaz and everything else as rus?
for i in range(len(labels)):
#    df_1000['labels'][i]=0
    if(labels[0][i].endswith(tuple(ext))!=True):
            print(df_1000['text'][i])
            print(labels[0][i])

РАПОРТ СТ.СЛЕДОВАТЕЛЯ СД МВД ДЖАКСЫБАЕВА Е., ПО ФАКТУ МОШЕННИЧЕСТВА СО СТОРОНЫ НЕИЗВЕСТНЫХ НА ТЕРРИТОРИИ Г.АЛМАТЫ  С ЗАВЛАДЕНИЕМ А/М «Toyota Land Cruiser Prado»
__label__tur
НСД
__label__cmn
НСД
__label__cmn
№155154031000299 санды кылмыстык істен Сланов Талгат жане аныкталмаган тулгаларга катысты кужаттар болектеп алынды
__label__tat
155154031000258 санды СДТБТ алдын ала тергеу амалдарын жургізу кезынде косымша аныкталганы. 2015 жылы  акпан айынын 05-нен 06-на караган туні сагат 03-00 шамасында 1986 ж.т. Баймешов Калдыкожа Хасанович касында белгысыз тулгалармен бузакылык жасау максатында Сарыагаш ауданы, Жарты тобе а/о, Ынтымак ауылы Бескорган кошесі №40 уйдін алдында тергеумен аныкталмаган таспен аталган уйдін тургыны 11.10.1957 ж.т. Махсумов Расулжон Мансуровичтін басын урып сдс-корытындысы бойынша женіл дарежелі зиян келтірген.
__label__tat


In [133]:
df_1000.assign(langs=labels)

,C1,C2,C3,C4,C5,C6,C7,text,num,langs
0,0,22.01.15,150000011000008,1971,Ищанов А. и Угай А. 12 декабря 2011 года обрат...,1900003,Мошенничество,Ищанов А. и Угай А. 12 декабря 2011 года обрат...,0.0,__label__rus
1,0,05.02.15,150000011000013,1971,"""УТВЕРЖДАЮ""\r\nНачальник 2-го Департамента\r\n...",1900003,Мошенничество,"""УТВЕРЖДАЮ"" Начальник 2-го Департамента Генера...",0.0,__label__rus
2,0,09.02.15,150000011000014,1971,08.06.2013г. в Талгарское РУВД ДВД Алматинской...,1900001,Мошенничество,08.06.2013г. в Талгарское РУВД ДВД Алматинской...,0.0,__label__rus
3,0,26.02.15,150000011000019,1971,22.04.2011г. в ДБЭКП по Восточно-Казахстанской...,1900002,Мошенничество,22.04.2011г. в ДБЭКП по Восточно-Казахстанской...,0.0,__label__rus
4,0,19.05.15,150000011000027,1971,Изучением материалов по фактам якобы незаконно...,1900001,Мошенничество,Изучением материалов по фактам якобы незаконно...,0.0,__label__rus
5,0,03.01.15,150000031000001,1971,24.06.205 года в г.Анкара гражданин Турции Хад...,1900004,Мошенничество,24.06.205 года в г.Анкара гражданин Турции Хад...,0.0,__label__rus
6,0,09.01.15,150000031000002,1971,по факту мошеничества в отношении Боштаевой Г....,1900001,Мошенничество,по факту мошеничества в отношении Боштаевой Г....,0.0,__label__rus
7,0,30.01.15,150000031000011,1971,ПО ФАКТУ МОШЕННИЧЕСТВА СО СТОРОНЫ НЕИЗВЕСТНЫХ ...,1900002,Мошенничество,ПО ФАКТУ МОШЕННИЧЕСТВА СО СТОРОНЫ НЕИЗВЕСТНЫХ ...,0.0,__label__rus
8,0,05.02.15,150000031000012,1971,ПО ФАКТУ МОШЕНИЧЕСТВА СО СТОРОНЫ АБДУЛГАФАРОВА...,1900004,Мошенничество,ПО ФАКТУ МОШЕНИЧЕСТВА СО СТОРОНЫ АБДУЛГАФАРОВА...,0.0,__label__rus
9,0,11.02.15,150000031000015,1971,ПО ФАКТУ МОШЕННИЧЕСКИХ ДЕЙСТВИЙ СО СТОРОНЫ НЕИ...,1940003,Вымогательство,ПО ФАКТУ МОШЕННИЧЕСКИХ ДЕЙСТВИЙ СО СТОРОНЫ НЕИ...,0.0,__label__rus


In [140]:
print(type(df_1000['text']))
print(type(labels))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [137]:
df_1000['text'].tail()

995    ПОГОРЕЛОВОЙ ЕВ 1968ГР ПРОЖ№ МИРА 27/20 ЧТО В Т...
996    ПРИЯТЬ  МЕРЫ В ОТНОШЕНИИ  МАКСИМОВА А И СЫЗДЫК...
997    ЛУКАШЕНКО И В 1975ГР КРАСНЫЙ-ЯР УЛ.ВАЛИХАНОВА ...
998    БАВЙЗАКОВА Т Т 1989ГР ПРОЖ.КАЛИНИНА 59/2 ЗА ДО...
999    ПРИНЯТЬ  МЕРВ ПО РОЗЫСКУ СОТОВОГО ТЕЛЕФОНАСАМС...
Name: text, dtype: object

In [34]:
print(labels)

                 0
0     __label__rus
1     __label__rus
2     __label__rus
3     __label__rus
4     __label__rus
5     __label__rus
6     __label__rus
7     __label__rus
8     __label__rus
9     __label__rus
10    __label__rus
11    __label__rus
12    __label__rus
13    __label__rus
14    __label__rus
15    __label__rus
16    __label__rus
17    __label__rus
18    __label__rus
19    __label__rus
20    __label__rus
21    __label__rus
22    __label__rus
23    __label__rus
24    __label__rus
25    __label__rus
26    __label__rus
27    __label__rus
28    __label__rus
29    __label__rus
...            ...
9970  __label__rus
9971  __label__rus
9972  __label__rus
9973  __label__rus
9974  __label__rus
9975  __label__rus
9976  __label__rus
9977  __label__rus
9978  __label__rus
9979  __label__rus
9980  __label__rus
9981  __label__rus
9982  __label__rus
9983  __label__rus
9984  __label__rus
9985  __label__rus
9986  __label__rus
9987  __label__rus
9988  __label__rus
9989  __label__rus
9990  __labe

In [33]:
print(df_1000['labels'])

30000    NaN
30001    NaN
30002    NaN
30003    NaN
30004    NaN
30005    NaN
30006    NaN
30007    NaN
30008    NaN
30009    NaN
30010    NaN
30011    NaN
30012    NaN
30013    NaN
30014    NaN
30015    NaN
30016    NaN
30017    NaN
30018    NaN
30019    NaN
30020    NaN
30021    NaN
30022    NaN
30023    NaN
30024    NaN
30025    NaN
30026    NaN
30027    NaN
30028    NaN
30029    NaN
        ... 
39970    NaN
39971    NaN
39972    NaN
39973    NaN
39974    NaN
39975    NaN
39976    NaN
39977    NaN
39978    NaN
39979    NaN
39980    NaN
39981    NaN
39982    NaN
39983    NaN
39984    NaN
39985    NaN
39986    NaN
39987    NaN
39988    NaN
39989    NaN
39990    NaN
39991    NaN
39992    NaN
39993    NaN
39994    NaN
39995    NaN
39996    NaN
39997    NaN
39998    NaN
39999    NaN
Name: labels, Length: 10000, dtype: object


In [32]:
df_1000[df_1000['labels'].str.contains('ac')]

ValueError: cannot index with vector containing NA / NaN values

In [ ]:
#get keywords from df_zhyl_only


In [49]:
import re

#str = "123456790abcdefABCDEF!@#$%^&*()_+<>?,./"
for i in range(20):
    print(df_1000['text'].values[i])
    result = re.sub(r'[^a-zA-Zа-яА-Я\n]', " ", df_1000['text'].values[i])
    print(result)

#df = df.groupby('domain')['ID'].nunique()

#print(df_kaz['text'].values[1])
#print(type(df_kaz['text'][:1]))
#regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string


по факту повреждение окна магазина Гульзия расположенного в п.Нокина.
по факту повреждение окна магазина Гульзия расположенного в п Нокина 
по факту утери сотового телефона марки самсунг
по факту утери сотового телефона марки самсунг
что 25.12.2015 года прмиерно 18-00 часов в с.каргалинское на маршрутном автобусе 19 утеряла сотовый телефон марки лж
что            года прмиерно       часов в с каргалинское на маршрутном автобусе    утеряла сотовый телефон марки лж
с.Каргалинское возле АЗС Ростанефть драка
с Каргалинское возле АЗС Ростанефть драка
ж\м Рауан н\л открыто похители сумку
ж м Рауан н л открыто похители сумку
п.Кирпичный ул.Лиманная 1 н\л проник через забор
п Кирпичный ул Лиманная   н л проник через забор
с.Каргалинское по ул.Геологическая возле д.17 с а/м ВАЗ 2107 г/н 265 ETA 04 совершили кражу аккумулятора.
с Каргалинское по ул Геологическая возле д    с а м ВАЗ      г н     ETA    совершили кражу аккумулятора 
обратилась Кушкинбаева А.А. 1984 г.р.прож.г.Актобе с/к Лекоровод

In [88]:
word_tokenize(result)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\931019350105/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\931019350105\\AppData\\Local\\Continuum\\anaconda3\\nltk_data'
    - 'C:\\Users\\931019350105\\AppData\\Local\\Continuum\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\931019350105\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************

In [ ]:
def get_words(text):
    for i in range(len(text)):
        

In [21]:
print(df_kaz['C5'].head(20))
print(df_kaz['text'].head(20))

32      27.09.2013 года неустановленные лица, путем об...
35      12.10.2013 года неустановленные лица, путем об...
140     №151900041000001 санды ќылмыстыќ іс бойынша со...
141     №151900041000001 санды ќылмыстыќ іс бойынша со...
142     №151900041000001 санды ќылмыстыќ іс бойынша со...
143     №14751503104098 санды ќылмыстыќ іс бойынша сот...
144     №14751503104098 санды ќылмыстыќ іс бойынша сот...
173     №155154031000299 санды кылмыстык істен Сланов ...
174     155154031000258 санды СДТБТ алдын ала тергеу а...
414     Зансыз арекеттер фактысы бойынша белгісіз тулг...
449     28.06.2014 года УБН ДВД Акмолинской области  с...
1372    В ДЕЖУРНУЮ ЧАСТЬ УВД г.КОКШЕТАУ ПОСТУПИЛО ЗАЯВ...
3163    Дата-время совершения:22.06.2015 21:32:56 Т/С ...
3709    ПРИНЯТЬ МЕРЫ К РОЗЫСКУ СОТОВОГО ТЕЛЕФОНА МАРКИ...
5192    Дата-время совершения:03.11.2015 19:53:46 Т/С ...
5277    РАПОРТ  СО УВД г.Кокшетау старший лейтенант по...
5404    В  Д/Ч УВД С ЗАЯВЛ-ЕМ ОБР ГР  ЕСЛЯМБЕК А Ј  19...
5685    В Д/Ч 

In [17]:
print(len(df2))
print(len(df_kaz))
print(len(df_kaz)/(len(df2)-len(df_kaz)))

1021715
169617
0.19905808956246818


In [9]:
print(df_kaz['C5'])

NameError: name 'df_kaz' is not defined

In [25]:
X=df2["C5"]
y=df2["C7"]


In [28]:
X.head()
y.head()


0    Мошенничество
1    Мошенничество
2    Мошенничество
3    Мошенничество
4    Мошенничество
Name: C7, dtype: object

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [46]:
print(len(y_train))
print(y_train.value_counts(0))

print(y_train.value_counts(1))

print(len(y_test))
print(y_test.value_counts(0))


print(y_test.value_counts(1))
#baselineScore=max(y.value_counts())/len(y)
#print(baselineScore)

919543
Кража             629304
Мошенничество     132571
Хулиганство        75729
Наркотики          42443
Грабеж             33975
Вымогательство      3000
Разбой              2521
Name: C7, dtype: int64
Кража             0.684366
Мошенничество     0.144171
Хулиганство       0.082355
Наркотики         0.046157
Грабеж            0.036948
Вымогательство    0.003262
Разбой            0.002742
Name: C7, dtype: float64
102172
Кража             70049
Мошенничество     14743
Хулиганство        8293
Наркотики          4656
Грабеж             3817
Вымогательство      332
Разбой              282
Name: C7, dtype: int64
Кража             0.685599
Мошенничество     0.144296
Хулиганство       0.081167
Наркотики         0.045570
Грабеж            0.037359
Вымогательство    0.003249
Разбой            0.002760
Name: C7, dtype: float64
